In [1]:
!python --version

Python 3.6.6 :: Anaconda, Inc.


In [2]:
import pkg_resources
dists = [d for d in pkg_resources.working_set]
print(dists)

[zipp 2.2.0 (/opt/conda/lib/python3.6/site-packages), zict 2.0.0 (/opt/conda/lib/python3.6/site-packages), yellowbrick 1.1 (/opt/conda/lib/python3.6/site-packages), yarl 1.4.2 (/opt/conda/lib/python3.6/site-packages), yapf 0.28.0 (/opt/conda/lib/python3.6/site-packages), xxhash 1.3.0 (/opt/conda/lib/python3.6/site-packages), xvfbwrapper 0.2.9 (/opt/conda/lib/python3.6/site-packages), xmltodict 0.12.0 (/opt/conda/lib/python3.6/site-packages), xlwt 1.3.0 (/opt/conda/lib/python3.6/site-packages), XlsxWriter 1.2.8 (/opt/conda/lib/python3.6/site-packages), xlrd 1.2.0 (/opt/conda/lib/python3.6/site-packages), xgboost 1.0.2 (/opt/conda/lib/python3.6/site-packages), xarray 0.15.1 (/opt/conda/lib/python3.6/site-packages), wurlitzer 2.0.0 (/opt/conda/lib/python3.6/site-packages), wrapt 1.11.2 (/opt/conda/lib/python3.6/site-packages), wordsegment 1.3.1 (/opt/conda/lib/python3.6/site-packages), wordcloud 1.6.0 (/opt/conda/lib/python3.6/site-packages), Wordbatch 1.4.4 (/opt/conda/lib/python3.6/site

In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
sub=pd.read_csv("../input/predicttheglobalspreadofcovid19/SampleSubmission.csv")
sub.head()
new=sub['Territory X Date'].str.split(" X ", n = 1, expand = True) 
sub["Country_Region"]=new[0]
sub["Date"]=new[1]

test=pd.read_csv("../input/covid19globalforecastingweek4/test.csv")
test.head()
test2=test.groupby(['Province_State','Country_Region']).size().reset_index()
test2.columns=['Province_State','Country_Region','Count']
sub=sub.merge(test2,how="left",on="Country_Region")
sub.drop(['Count'],1,inplace=True)
sub['Date'] = pd.to_datetime(sub['Date'], format='%m/%d/%y')
sub['ForecastId']=sub.index+1
sub.head()

,Territory X Date,target,Country_Region,Date,Province_State,ForecastId
0,Afghanistan X 3/6/20,0,Afghanistan,2020-03-06,NaN,1
1,Afghanistan X 3/7/20,0,Afghanistan,2020-03-07,NaN,2
2,Afghanistan X 3/8/20,0,Afghanistan,2020-03-08,NaN,3
3,Afghanistan X 3/9/20,0,Afghanistan,2020-03-09,NaN,4
4,Afghanistan X 3/10/20,0,Afghanistan,2020-03-10,NaN,5


In [4]:
%matplotlib inline
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', 99)
pd.set_option('display.max_rows', 99)
import os
import numpy as np
from matplotlib import pyplot as plt
from tqdm import tqdm
import datetime as dt
import xgboost as xgb
from sklearn import preprocessing
from scipy.stats import gmean

In [5]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
import seaborn as sns
sns.set_palette(sns.color_palette('tab20', 20))

import plotly.express as px
import plotly.graph_objects as go

In [6]:

DATA_PATH = '../input/covid19-metadata'
DATEFORMAT = '%Y-%m-%d'


def get_comp_data(COMP):
    train = pd.read_csv(f'{COMP}/train.csv')
    test = sub.copy()#pd.read_csv(f'{COMP}/test.csv')
    test['Date']=test['Date'].astype(str)
    submission = pd.read_csv(f'{COMP}/submission.csv')
    print(train.shape, test.shape, submission.shape)
    train['Country_Region'] = train['Country_Region'].str.replace(',', '')
    test['Country_Region'] = test['Country_Region'].str.replace(',', '')

    train['Location'] = train['Country_Region'] + '-' + train['Province_State'].fillna('')

    test['Location'] = test['Country_Region'] + '-' + test['Province_State'].fillna('')

    train['LogConfirmed'] = to_log(train.ConfirmedCases)
    train['LogFatalities'] = to_log(train.Fatalities)
    train = train.drop(columns=['Province_State'])
    test = test.drop(columns=['Province_State'])

    country_codes = pd.read_csv(f'{DATA_PATH}/country_codes.csv', keep_default_na=False)
    train = train.merge(country_codes, on='Country_Region', how='left')
    test = test.merge(country_codes, on='Country_Region', how='left')

    train['DateTime'] = pd.to_datetime(train['Date'])
    test['DateTime'] = pd.to_datetime(test['Date'])

    train = train.sort_values(by='Date')
    test = test.sort_values(by='Date')

    train = train.fillna('#N/A')
    test = test.fillna('#N/A')

    return train, test, submission


def process_each_location(df):
    dfs = []
    for loc, df in df.groupby('Location'):
        df = df.sort_values(by='Date')
        df['Fatalities'] = df['Fatalities'].cummax()
        df['ConfirmedCases'] = df['ConfirmedCases'].cummax()
        df['LogFatalities'] = df['LogFatalities'].cummax()
        df['LogConfirmed'] = df['LogConfirmed'].cummax()
        df['LogConfirmedNextDay'] = df['LogConfirmed'].shift(-1)
        df['ConfirmedNextDay'] = df['ConfirmedCases'].shift(-1)
        df['DateNextDay'] = df['Date'].shift(-1)
        df['LogFatalitiesNextDay'] = df['LogFatalities'].shift(-1)
        df['FatalitiesNextDay'] = df['Fatalities'].shift(-1)
        df['LogConfirmedDelta'] = df['LogConfirmedNextDay'] - df['LogConfirmed']
        df['ConfirmedDelta'] = df['ConfirmedNextDay'] - df['ConfirmedCases']
        df['LogFatalitiesDelta'] = df['LogFatalitiesNextDay'] - df['LogFatalities']
        df['FatalitiesDelta'] = df['FatalitiesNextDay'] - df['Fatalities']
        dfs.append(df)
    return pd.concat(dfs)


def add_days(d, k):
    return dt.datetime.strptime(d, DATEFORMAT) + dt.timedelta(days=k)


def to_log(x):
    return np.log(x + 1)


def to_exp(x):
    return np.exp(x) - 1


def create_features(train_set):
    dfs = []
    for loc, df in train_set.groupby('Location'):
        df = df.sort_values(by='Date').copy()
        df['f_lc_7d'] = df['LogConfirmed'].shift(7)
        df['f_lf_7d'] = df['LogFatalities'].shift(7)
        df['f_lc_3d'] = df['LogConfirmed'].shift(3)
        df['f_lf_3d'] = df['LogFatalities'].shift(3)
        df['f_lc_1d'] = df['LogConfirmed'].shift(1)
        df['f_lf_1d'] = df['LogFatalities'].shift(1)
        df['f_lc_0d'] = df['LogConfirmed']
        df['f_lf_0d'] = df['LogFatalities']
        df['f_fc_rate'] = np.clip((to_exp(df['LogFatalities']) + 1) / (to_exp(df['LogConfirmed']) + 1), 0, 0.15)
        dfs.append(df)
    dfs = pd.concat(dfs)
    dfs['d_lc_7d'] = dfs['f_lc_0d'] - dfs['f_lc_7d']
    dfs['d_lf_7d'] = dfs['f_lf_0d'] - dfs['f_lf_7d']
    dfs['d_lc_3d'] = dfs['f_lc_3d'] - dfs['f_lc_3d']
    dfs['d_lf_3d'] = dfs['f_lf_3d'] - dfs['f_lf_3d']
    dfs['d_lc_1d'] = dfs['f_lc_0d'] - dfs['f_lc_1d']
    dfs['d_lf_1d'] = dfs['f_lf_0d'] - dfs['f_lf_1d']
    return dfs


In [7]:

COMP = '../input/covid19globalforecastingweek4'
start = dt.datetime.now()
train, test, submission = get_comp_data(COMP)
train.shape, test.shape, submission.shape
train.head(2)
test.head(2)

TRAIN_START = train.Date.min()
TEST_START = test.Date.min()
TRAIN_END = train.Date.max()
TEST_END = test.Date.max()
print(TRAIN_START, TRAIN_END, TEST_START, TEST_END)

(35995, 6) (25286, 6) (13459, 3)


((35995, 13), (25286, 11), (13459, 3))

,Id,Country_Region,Date,ConfirmedCases,Fatalities,Location,LogConfirmed,LogFatalities,country_iso_code_2,country_iso_code_3,continent,geo_region,DateTime
0,1,Afghanistan,2020-01-22,0.0,0.0,Afghanistan-,0.0,0.0,AF,AFG,Asia,Southern Asia,2020-01-22
23115,23116,Portugal,2020-01-22,0.0,0.0,Portugal-,0.0,0.0,PT,PRT,Europe,Southern Europe,2020-01-22


,Territory X Date,target,Country_Region,Date,ForecastId,Location,country_iso_code_2,country_iso_code_3,continent,geo_region,DateTime
0,Afghanistan X 3/6/20,0,Afghanistan,2020-03-06,1,Afghanistan-,AF,AFG,Asia,Southern Asia,2020-03-06
3666,Cameroon X 3/6/20,0,Cameroon,2020-03-06,3667,Cameroon-,CM,CMR,Africa,Middle Africa,2020-03-06


2020-01-22 2020-05-15 2020-03-06 2020-06-07


In [8]:
train_clean = process_each_location(train)

print('train cleaned', train_clean.shape)

train_clean = train_clean[[
    'Location', 'Date', 'continent',
    'LogConfirmed', 'LogFatalities',
    'LogConfirmedDelta', 'LogFatalitiesDelta'
]]

continent_encoder = preprocessing.LabelEncoder()
train_clean['f_continent'] = continent_encoder.fit_transform(train_clean.continent)

train_features = create_features(train_clean)
train_features.head()

train cleaned (35995, 22)


,Location,Date,continent,LogConfirmed,LogFatalities,LogConfirmedDelta,LogFatalitiesDelta,f_continent,f_lc_7d,f_lf_7d,f_lc_3d,f_lf_3d,f_lc_1d,f_lf_1d,f_lc_0d,f_lf_0d,f_fc_rate,d_lc_7d,d_lf_7d,d_lc_3d,d_lf_3d,d_lc_1d,d_lf_1d
0,Afghanistan-,2020-01-22,Asia,0.0,0.0,0.0,0.0,3,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.15,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan-,2020-01-23,Asia,0.0,0.0,0.0,0.0,3,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.15,NaN,NaN,NaN,NaN,0.0,0.0
2,Afghanistan-,2020-01-24,Asia,0.0,0.0,0.0,0.0,3,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.15,NaN,NaN,NaN,NaN,0.0,0.0
3,Afghanistan-,2020-01-25,Asia,0.0,0.0,0.0,0.0,3,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.15,NaN,NaN,0.0,0.0,0.0,0.0
4,Afghanistan-,2020-01-26,Asia,0.0,0.0,0.0,0.0,3,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.15,NaN,NaN,0.0,0.0,0.0,0.0


In [9]:
def predict(min_child_weight, eta, colsample_bytree, max_depth, subsample,
           NROUND, PRECISION, DECAY, WEIGHT_NORM, MIN_DATE):
    train, test, submission = get_comp_data(COMP)
    train_clean = process_each_location(train)
    print('train cleaned', train_clean.shape)

    train_clean = train_clean[[
        'Location', 'Date', 'continent',
        'LogConfirmed', 'LogFatalities',
        'LogConfirmedDelta', 'LogFatalitiesDelta'
    ]]

    continent_encoder = preprocessing.LabelEncoder()
    train_clean['f_continent'] = continent_encoder.fit_transform(train_clean.continent)

    train_features = create_features(train_clean)

    VALID_CUTOFF = TRAIN_END

    features = [
        'f_lc_7d', 'f_lf_7d', 'f_lc_3d', 'f_lf_3d', 'f_continent',
        'f_lc_1d', 'f_lf_1d', 'f_lc_0d', 'f_lf_0d', 'f_fc_rate'
    ]
    diff_features = ['d_lc_7d', 'd_lf_7d', 'd_lc_3d', 'd_lf_3d', 'd_lc_1d', 'd_lf_1d']
    features = features + diff_features
    print(f'{len(features)} features: ')
    print(features)
    
    fix = {
    'lambda': 1., 'nthread': 3, 'booster': 'gbtree',
    'silent': 1, 'eval_metric': 'rmse',
    'objective': 'reg:squarederror'}
    config = dict(
        min_child_weight=min_child_weight,
        eta=eta, colsample_bytree=colsample_bytree,
        max_depth=max_depth, subsample=subsample)
    config.update(fix)

    Xtr = train_features[(train_features.Date >= MIN_DATE) & (train_features.Date < VALID_CUTOFF)].copy()
    Xtr['days'] = -(pd.to_datetime(train_features.Date) - dt.datetime.strptime(VALID_CUTOFF, DATEFORMAT)).dt.days
    
    print(Xtr.shape)
    print(config)
    print(PRECISION, NROUND, DECAY, WEIGHT_NORM)

    def weighting(days):
        return 1. / days ** WEIGHT_NORM

    dtrain_lc = xgb.DMatrix(Xtr[features].round(PRECISION), label=Xtr.LogConfirmedDelta, weight=weighting(Xtr.days))
    dtrain_lf = xgb.DMatrix(Xtr[features].round(PRECISION), label=Xtr.LogFatalitiesDelta, weight=weighting(Xtr.days))

    model_lc = xgb.train(config, dtrain_lc, NROUND, evals=[(dtrain_lc, 'train-lc')], verbose_eval=100)
    model_lf = xgb.train(config, dtrain_lf, NROUND, evals=[(dtrain_lf, 'train-lf')], verbose_eval=100)

    # Predict

    predictions = Xtr.copy()
    predictions = train_features[(train_features.Date >= MIN_DATE) & (train_features.Date <= VALID_CUTOFF)].copy()
    predictions.LogConfirmedDelta = np.nan
    predictions.LogFatalitiesDelta = np.nan

    for i, d in enumerate(pd.date_range(VALID_CUTOFF, add_days(TEST_END, 1))):
        last_day = str(d).split(' ')[0]
        next_day = dt.datetime.strptime(last_day, DATEFORMAT) + dt.timedelta(days=1)
        next_day = next_day.strftime(DATEFORMAT)

        p_next_day = predictions[predictions.Date == last_day].copy()
        p_next_day.Date = next_day
        p_next_day['plc'] = model_lc.predict(xgb.DMatrix(p_next_day[features].round(PRECISION)))
        p_next_day['plf'] = model_lf.predict(xgb.DMatrix(p_next_day[features].round(PRECISION)))

        p_next_day.LogConfirmed = p_next_day.LogConfirmed + np.clip(p_next_day['plc'], 0, None) * DECAY ** i
        p_next_day.LogFatalities = p_next_day.LogFatalities + np.clip(p_next_day['plf'], 0, None) * DECAY ** i

        predictions = pd.concat([predictions, p_next_day], sort=True)
        predictions = create_features(predictions)

    predictions['PC'] = to_exp(predictions.LogConfirmed)
    predictions['PF'] = to_exp(predictions.LogFatalities)
    return predictions

In [10]:
decay = 0.99
prediction_1 = predict(min_child_weight=5, eta=0.01, colsample_bytree=0.8, max_depth=5, subsample=0.9,
           NROUND=800, PRECISION=2, DECAY=decay, WEIGHT_NORM=0.25, MIN_DATE='2020-03-22')
prediction_2 = predict(min_child_weight=7, eta=0.01, colsample_bytree=0.7, max_depth=6, subsample=0.8,
           NROUND=1000, PRECISION=2, DECAY=decay, WEIGHT_NORM=0.23, MIN_DATE='2020-03-15')
prediction_3 = predict(min_child_weight=3, eta=0.01, colsample_bytree=0.6, max_depth=7, subsample=0.7,
           NROUND=1200, PRECISION=2, DECAY=decay, WEIGHT_NORM=0.2, MIN_DATE='2020-03-08')
prediction_4 = predict(min_child_weight=3, eta=0.011, colsample_bytree=0.75, max_depth=10, subsample=0.6,
           NROUND=1200, PRECISION=3, DECAY=decay, WEIGHT_NORM=0.15, MIN_DATE='2020-03-15')
prediction_5 = predict(min_child_weight=10, eta=0.008, colsample_bytree=0.75, max_depth=10, subsample=0.6,
           NROUND=1500, PRECISION=3, DECAY=decay, WEIGHT_NORM=0.2, MIN_DATE='2020-03-22')
prediction_6 = predict(min_child_weight=20, eta=0.01, colsample_bytree=0.7, max_depth=5, subsample=0.85,
           NROUND=1000, PRECISION=3, DECAY=decay, WEIGHT_NORM=0.3, MIN_DATE='2020-03-22')

(35995, 6) (25286, 6) (13459, 3)
train cleaned (35995, 22)
16 features: 
['f_lc_7d', 'f_lf_7d', 'f_lc_3d', 'f_lf_3d', 'f_continent', 'f_lc_1d', 'f_lf_1d', 'f_lc_0d', 'f_lf_0d', 'f_fc_rate', 'd_lc_7d', 'd_lf_7d', 'd_lc_3d', 'd_lf_3d', 'd_lc_1d', 'd_lf_1d']
(16902, 24)
{'min_child_weight': 5, 'eta': 0.01, 'colsample_bytree': 0.8, 'max_depth': 5, 'subsample': 0.9, 'lambda': 1.0, 'nthread': 3, 'booster': 'gbtree', 'silent': 1, 'eval_metric': 'rmse', 'objective': 'reg:squarederror'}
2 800 0.99 0.25
[0]	train-lc-rmse:0.45880
[100]	train-lc-rmse:0.18652
[200]	train-lc-rmse:0.10465
[300]	train-lc-rmse:0.08732
[400]	train-lc-rmse:0.08421
[500]	train-lc-rmse:0.08344
[600]	train-lc-rmse:0.08302
[700]	train-lc-rmse:0.08263
[799]	train-lc-rmse:0.08234
[0]	train-lf-rmse:0.46450
[100]	train-lf-rmse:0.18913
[200]	train-lf-rmse:0.10613
[300]	train-lf-rmse:0.08811
[400]	train-lf-rmse:0.08448
[500]	train-lf-rmse:0.08317
[600]	train-lf-rmse:0.08237
[700]	train-lf-rmse:0.08163
[799]	train-lf-rmse:0.08094
(

In [11]:
cols = ['Location', 'Date', 'PC', 'PF']
p12 = pd.merge(prediction_1[cols], prediction_2[cols], on=['Location', 'Date'], suffixes=['_1', '_2'])
p34 = pd.merge(prediction_3[cols], prediction_4[cols], on=['Location', 'Date'], suffixes=['_3', '_4'])
p56 = pd.merge(prediction_5[cols], prediction_6[cols], on=['Location', 'Date'], suffixes=['_5', '_6'])

preds = pd.merge(p12, p34, on=['Location', 'Date'])
preds = pd.merge(preds, p56, on=['Location', 'Date'])
preds.head()

c = preds.loc[preds.Date >= '2020-04-15'].corr()
c
fig = px.imshow(c)
fig.show()

,Location,Date,PC_1,PF_1,PC_2,PF_2,PC_3,PF_3,PC_4,PF_4,PC_5,PF_5,PC_6,PF_6
0,Afghanistan-,2020-03-22,40.0,1.0,40.0,1.0,40.0,1.0,40.0,1.0,40.0,1.0,40.0,1.0
1,Afghanistan-,2020-03-23,40.0,1.0,40.0,1.0,40.0,1.0,40.0,1.0,40.0,1.0,40.0,1.0
2,Afghanistan-,2020-03-24,74.0,1.0,74.0,1.0,74.0,1.0,74.0,1.0,74.0,1.0,74.0,1.0
3,Afghanistan-,2020-03-25,84.0,2.0,84.0,2.0,84.0,2.0,84.0,2.0,84.0,2.0,84.0,2.0
4,Afghanistan-,2020-03-26,94.0,4.0,94.0,4.0,94.0,4.0,94.0,4.0,94.0,4.0,94.0,4.0


,PC_1,PF_1,PC_2,PF_2,PC_3,PF_3,PC_4,PF_4,PC_5,PF_5,PC_6,PF_6
PC_1,1.000000,0.857408,0.999954,0.858562,0.999918,0.859023,0.999918,0.861034,0.999675,0.860133,0.999929,0.859674
PF_1,0.857408,1.000000,0.858025,0.999946,0.857319,0.999926,0.857317,0.999830,0.854177,0.999807,0.857490,0.999888
PC_2,0.999954,0.858025,1.000000,0.859215,0.999972,0.859670,0.999920,0.861670,0.999606,0.860772,0.999912,0.860322
PF_2,0.858562,0.999946,0.859215,1.000000,0.858516,0.999967,0.858509,0.999929,0.855332,0.999922,0.858664,0.999972
PC_3,0.999918,0.857319,0.999972,0.858516,1.000000,0.859010,0.999945,0.861005,0.999696,0.860094,0.999902,0.859635
PF_3,0.859023,0.999926,0.859670,0.999967,0.859010,1.000000,0.859008,0.999934,0.855849,0.999901,0.859135,0.999954
PC_4,0.999918,0.857317,0.999920,0.858509,0.999945,0.859008,1.000000,0.861040,0.999799,0.860138,0.999949,0.859666
PF_4,0.861034,0.999830,0.861670,0.999929,0.861005,0.999934,0.861040,1.000000,0.857914,0.999959,0.861169,0.999964
PC_5,0.999675,0.854177,0.999606,0.855332,0.999696,0.855849,0.999799,0.857914,1.000000,0.857060,0.999794,0.856520
PF_5,0.860133,0.999807,0.860772,0.999922,0.860094,0.999901,0.860138,0.999959,0.857060,1.000000,0.860294,0.999970


In [12]:
pcs = ['PC_1', 'PC_2', 'PC_3', 'PC_4', 'PC_5', 'PC_6']
pfs = ['PF_1', 'PF_2', 'PF_3', 'PF_4', 'PF_5', 'PF_6']
preds['PC'] = to_exp(to_log(preds[pcs]).mean(axis=1))
preds['PF'] = to_exp(to_log(preds[pfs]).mean(axis=1))
preds.tail()

,Location,Date,PC_1,PF_1,PC_2,PF_2,PC_3,PF_3,PC_4,PF_4,PC_5,PF_5,PC_6,PF_6,PC,PF
25035,Zimbabwe-,2020-06-05,78.236425,6.101347,78.725055,5.812992,80.431334,5.924536,88.506660,6.005013,89.279335,6.176099,80.701717,5.948155,82.527796,5.993684
25036,Zimbabwe-,2020-06-06,80.332726,6.229999,80.565164,5.913251,82.403785,6.028174,90.737086,6.150089,91.807497,6.311797,82.757388,6.071357,84.641544,6.116234
25037,Zimbabwe-,2020-06-07,82.433956,6.345268,82.453435,5.995490,84.253744,6.099131,92.914385,6.384454,94.187404,6.485417,84.755245,6.200464,86.702533,6.249716
25038,Zimbabwe-,2020-06-08,84.574648,6.461194,84.442951,6.076523,86.064315,6.173893,95.163921,6.438511,96.533141,6.589240,86.756739,6.298216,88.787883,6.337487
25039,Zimbabwe-,2020-06-09,86.740964,6.556214,86.379566,6.155444,87.958501,6.253067,97.409627,6.505150,99.022992,6.647936,88.913909,6.397566,90.930723,6.417243


In [13]:
top_locations = preds[preds.Date == TRAIN_END].sort_values(by='PF', ascending=False).Location.values[:25]
fig3 = px.line(preds[preds.Location.isin(top_locations)],
               x='Date', y='PC', color='Location')
_ = fig3.update_layout(
    yaxis_type="log",
    title_text=f'COVID-19 Predicted Cumulative Confirmed Cases by Location [Updated: {TRAIN_END}]'
)
fig3.show()

In [14]:
top_locations = preds[preds.Date == TRAIN_END].sort_values(by='PF', ascending=False).Location.values[:25]
fig3 = px.line(preds[preds.Location.isin(top_locations)],
               x='Date', y='PF', color='Location')
_ = fig3.update_layout(
    yaxis_type="log",
    title_text=f'COVID-19 Predicted Cumulative Deaths by Location [Updated: {TRAIN_END}]'
)
fig3.show()

In [15]:
total = preds.groupby('Date')[['PC', 'PF'] + pcs + pfs].sum().reset_index()
total.tail()
fig2 = px.line(pd.melt(total, id_vars=['Date']), x='Date', y='value', color='variable')
_ = fig2.update_layout(
    yaxis_type="log",
    title_text=f'COVID-19 Cumulative Prediction Total [Updated: {TRAIN_END}]'
)
fig2.show()

,Date,PC,PF,PC_1,PC_2,PC_3,PC_4,PC_5,PC_6,PF_1,PF_2,PF_3,PF_4,PF_5,PF_6
75,2020-06-05,6.067363e+06,406905.903468,6.125751e+06,6.065644e+06,6.061845e+06,6.070101e+06,6.058462e+06,6.028639e+06,413098.038630,407300.105245,408863.236095,405349.153162,401272.811672,405959.689828
76,2020-06-06,6.112766e+06,410616.859048,6.176445e+06,6.111283e+06,6.107270e+06,6.115259e+06,6.101503e+06,6.071612e+06,417191.690167,410853.894235,412771.219516,409086.669603,404628.702638,409606.837078
77,2020-06-07,6.155906e+06,414252.125460,6.223454e+06,6.155189e+06,6.150735e+06,6.158255e+06,6.142117e+06,6.112956e+06,421159.700095,414349.446019,416600.547450,412819.277026,407891.541374,413159.872578
78,2020-06-08,6.197327e+06,417805.530592,6.268344e+06,6.197754e+06,6.192712e+06,6.200605e+06,6.179894e+06,6.152388e+06,425066.107719,417692.628145,420391.412064,416450.341314,411099.236108,416631.021151
79,2020-06-09,6.237077e+06,421269.978891,6.311679e+06,6.238983e+06,6.233202e+06,6.239980e+06,6.216165e+06,6.190630e+06,428932.557232,420966.398599,424077.609248,419968.298293,414197.115039,420003.335084


## Create submission

In [16]:
my_submission = test.copy()
my_submission = my_submission.merge(preds)
my_submission['ConfirmedCases'] = my_submission['PC']
my_submission['Fatalities'] = my_submission['PF']
my_submission.shape
my_submission.head()

(16458, 27)

,Territory X Date,target,Country_Region,Date,ForecastId,Location,country_iso_code_2,country_iso_code_3,continent,geo_region,DateTime,PC_1,PF_1,PC_2,PF_2,PC_3,PF_3,PC_4,PF_4,PC_5,PF_5,PC_6,PF_6,PC,PF,ConfirmedCases,Fatalities
0,Israel X 3/22/20,0,Israel,2020-03-22,14211,Israel-,IL,ISR,Asia,Western Asia,2020-03-22,883.0,1.0,883.0,1.0,883.0,1.0,883.0,1.0,883.0,1.0,883.0,1.0,883.0,1.0,883.0,1.0
1,Croatia X 3/22/20,0,Croatia,2020-03-22,8759,Croatia-,HR,HRV,Europe,Southern Europe,2020-03-22,254.0,1.0,254.0,1.0,254.0,1.0,254.0,1.0,254.0,1.0,254.0,1.0,254.0,1.0,254.0,1.0
2,France X 3/22/20,0,France,2020-03-22,11165,France-Reunion,FR,FRA,Europe,Western Europe,2020-03-22,64.0,0.0,64.0,0.0,64.0,0.0,64.0,0.0,64.0,0.0,64.0,0.0,64.0,0.0,64.0,0.0
3,Suriname X 3/22/20,0,Suriname,2020-03-22,22577,Suriname-,SR,SUR,Americas,South America,2020-03-22,5.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0,5.0,0.0
4,Peru X 3/22/20,0,Peru,2020-03-22,19381,Peru-,PE,PER,Americas,South America,2020-03-22,363.0,5.0,363.0,5.0,363.0,5.0,363.0,5.0,363.0,5.0,363.0,5.0,363.0,5.0,363.0,5.0


In [17]:
my_submission[['ForecastId', 'ConfirmedCases', 'Fatalities']].to_csv('submission.csv', index=False)
my_submission.groupby('Date')[['ConfirmedCases', 'Fatalities']].sum().reset_index().tail()

,Date,ConfirmedCases,Fatalities
73,2020-06-03,2.982433e+06,188542.521466
74,2020-06-04,3.011247e+06,190553.263273
75,2020-06-05,3.038767e+06,192516.320308
76,2020-06-06,3.064800e+06,194425.901529
77,2020-06-07,3.089240e+06,196271.948710


In [18]:
end = dt.datetime.now()
print('Finished', end, (end - start).seconds, 's')

Finished 2020-06-26 14:57:26.870332 699 s


In [19]:
my_submission['target']=my_submission['Fatalities'].fillna(0)
my_submission.sort_values(by='Territory X Date',inplace=True)
my_submission[['Territory X Date','target']].to_csv("submission_beluga.csv",index=False)
my_submission[['Territory X Date','target']].tail()

,Territory X Date,target
4393,Uruguay X 4/11/20,7.000000
4481,Uruguay X 4/12/20,7.000000
4750,Uruguay X 4/13/20,8.000000
4920,Uruguay X 4/14/20,8.000000
5234,Uruguay X 4/15/20,9.000000
...,...,...
15450,Zimbabwe X 6/3/20,5.780404
15674,Zimbabwe X 6/4/20,5.885938
15861,Zimbabwe X 6/5/20,5.993684
16134,Zimbabwe X 6/6/20,6.116234


In [20]:
my_submission[['Territory X Date','target']].head()

,Territory X Date,target
162,Afghanistan X 3/22/20,1.000000
275,Afghanistan X 3/23/20,1.000000
434,Afghanistan X 3/24/20,1.000000
668,Afghanistan X 3/25/20,2.000000
1051,Afghanistan X 3/26/20,4.000000
...,...,...
14532,Andorra X 5/29/20,53.263599
8971,Andorra X 5/3/20,45.000000
14591,Andorra X 5/30/20,53.558981
14967,Andorra X 5/31/20,53.853471


### Zindi Format

In [21]:
best=pd.read_csv("submission_beluga.csv")
test=pd.read_csv("../input/covid19globalforecastingweek4/test.csv")
best=best.merge(test,how="left",on="ForecastId")
best.head()

,ForecastId,ConfirmedCases,Fatalities,Province_State,Country_Region,Date
0,14211,883.0,1.0,NaN,NaN,NaN
1,8759,254.0,1.0,NaN,Romania,2020-05-01
2,11165,64.0,0.0,Massachusetts,US,2020-04-29
3,22577,5.0,0.0,NaN,NaN,NaN
4,19381,363.0,5.0,NaN,NaN,NaN


In [22]:
# best=pd.read_csv("submission_beluga.csv")
# print(best.shape)
# best.head()

In [23]:
sub=pd.read_csv("../input/predicttheglobalspreadofcovid19/SampleSubmission.csv")
print(sub.shape)
sub.head()

(19646, 2)


,Territory X Date,target
0,Afghanistan X 3/6/20,0
1,Afghanistan X 3/7/20,0
2,Afghanistan X 3/8/20,0
3,Afghanistan X 3/9/20,0
4,Afghanistan X 3/10/20,0


In [24]:
new=sub['Territory X Date'].str.split(" X ", n = 1, expand = True) 
sub["Country_Region"]=new[0]
sub["Date"]=new[1]
sub['Date'] = pd.to_datetime(sub['Date'], format='%m/%d/%y')
sub['Date']= sub['Date'].astype(str)
sub.head()


,Territory X Date,target,Country_Region,Date
0,Afghanistan X 3/6/20,0,Afghanistan,2020-03-06
1,Afghanistan X 3/7/20,0,Afghanistan,2020-03-07
2,Afghanistan X 3/8/20,0,Afghanistan,2020-03-08
3,Afghanistan X 3/9/20,0,Afghanistan,2020-03-09
4,Afghanistan X 3/10/20,0,Afghanistan,2020-03-10


In [25]:
best2=best.groupby(['Country_Region','Date'])['Fatalities'].sum().reset_index()
best2.loc[best2['Country_Region']=="US",'Country_Region']="United States of America (the)"
best2.loc[best2['Country_Region']=="Russia",'Country_Region']="Russian Federation (the)"
best2.loc[best2['Country_Region']=="United Kingdom",'Country_Region']="United Kingdom of Great Britain and Northern Ireland (the)"
best2.loc[best2['Country_Region']=="Iran",'Country_Region']="Iran (Islamic Republic of)"
best2.loc[best2['Country_Region']=="Venezuela",'Country_Region']="Venezuela (Bolivarian Republic of)"
best2.loc[best2['Country_Region']=="United Arab Emirates",'Country_Region']="United Arab Emirates (the)"
best2.loc[best2['Country_Region']=="Gambia",'Country_Region']="Gambia (the)"
best2.loc[best2['Country_Region']=="Netherlands",'Country_Region']="Netherlands (the)"
best2.loc[best2['Country_Region']=="Niger",'Country_Region']="Niger (the)"
best2.loc[best2['Country_Region']=="Tanzania",'Country_Region']="United Republic of Tanzania (the)"
best2.loc[best2['Country_Region']=="Vietnam",'Country_Region']="Viet Nam"
best2.loc[best2['Country_Region']=="Bahamas",'Country_Region']="Bahamas (the)"
best2.loc[best2['Country_Region']=="Bolivia",'Country_Region']="Bolivia (Plurinational State of)"
best2.loc[best2['Country_Region']=="Central African Republic",'Country_Region']="Central African Republic (the)"
best2.loc[best2['Country_Region']=="Congo",'Country_Region']="Congo (the)"
best2.loc[best2['Country_Region']=="Comoros",'Country_Region']="Comoros (the)"
best2.loc[best2['Country_Region']=="Korea, South",'Country_Region']="Republic of Korea (the)"
best2.loc[best2['Country_Region']=="Democratic Republic of the Congo",'Country_Region']="Democratic Republic of the Congo (the)"
best2.loc[best2['Country_Region']=="Dominican Republic",'Country_Region']="Dominican Republic (the)"
best2.loc[best2['Country_Region']=="Moldova",'Country_Region']="Republic of Moldova (the)"
best2.loc[best2['Country_Region']=="Sudan",'Country_Region']="Sudan (the)"
best2.loc[best2['Country_Region']=="Syria",'Country_Region']="Syrian Arab Republic (the)"

sub=sub.merge(best2,how="left",on=['Country_Region','Date'])
sub['target']=sub['Fatalities'].fillna(0)
sub[['Territory X Date','target']].to_csv("ma20200417_beluga.csv",index=False)
sub[['Territory X Date','target']].head()

,Territory X Date,target
0,Afghanistan X 3/6/20,0.0
1,Afghanistan X 3/7/20,0.0
2,Afghanistan X 3/8/20,0.0
3,Afghanistan X 3/9/20,0.0
4,Afghanistan X 3/10/20,0.0
